In [ ]:
import os
import subprocess
from Bio import SeqIO

genome_file = "example_genome/Escherichia_coli_K-12_MG1655.fasta"

## 1. predict orfs using prokka

def run_prokka(genome_file):
    os.makedirs("prokka_output", exist_ok=True)
    os.system(f"cp {genome_file} prokka_output/genome.fna")
    os.chdir("prokka_output")
    cmd = f"prokka --kingdom Bacteria genome.fna --noanno" # change Bacteria to Archaea if needed
    subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    os.chdir("..")

run_prokka(genome_file)

In [2]:
## 2. extract upper stream of orfs
from glob import glob

prokka_gbk_file = glob("prokka_output/PROKKA_*/*.gbk")[0]
putative_promoters = []
with open(prokka_gbk_file) as input_handle:
    for record in SeqIO.parse(input_handle, "genbank"):
        seq = record.seq
        for feature in record.features:
            type = feature.type
            upper_seq_160bp = None
            if type == "CDS":
                if feature.strand == 1 and feature.location.start > 160:
                    upper_seq_160bp = record.seq[feature.location.start-160:feature.location.start]
                if feature.location.strand == -1 and feature.location.end < len(record.seq)-160:
                    upper_seq_160bp = record.seq[feature.location.end:feature.location.end+160]
                    upper_seq_160bp = upper_seq_160bp.reverse_complement()
                if upper_seq_160bp:
                    putative_promoters.append(upper_seq_160bp)
with open("prokka_output/putative_promoters.fasta", "w") as output_handle:
    for i, sequence in enumerate(putative_promoters):
        output_handle.write(f">seq_{i}\n{sequence}\n")

## 3. predict promoters using promotercalculator

os.makedirs("promotercalculator_output", exist_ok=True)
from Promoter_Calculator_v1_0 import Promoter_Calculator
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed

core_promoters = []
predicted_Tx_rates = []

calc = Promoter_Calculator()

def process_sequence(sequence):
    calc.run(sequence, TSS_range=[60, len(sequence)])
    output = calc.output()
    score_list = [result['Tx_rate'] for result in output['Forward_Predictions_per_TSS'].values()]
    # print(sequence, score_list)
    index = score_list.index(max(score_list))
    core_promoter = sequence[index:index + 60]
    return sequence, core_promoter, max(score_list)

results = Parallel(n_jobs=-1)(delayed(process_sequence)(sequence) for sequence in tqdm(putative_promoters))

for result in results:
    core_promoters.append(result[1])
    predicted_Tx_rates.append(result[2])

df = pd.DataFrame({'Putative_Promoter': [result[0] for result in results], 'Core_Promoter': core_promoters, 'Predicted_Tx_Rate': predicted_Tx_rates})
df.to_csv("promotercalculator_output/predicted_promoters.csv", index=False)

with open("promotercalculator_output/predicted_promoters_strong.fasta", "w") as output_handle:
    for index, row in df.sort_values("Predicted_Tx_Rate",ascending=False).head(len(df)//5).iterrows():
        output_handle.write(f">{index}|ptx={row['Predicted_Tx_Rate']}\n{row['Core_Promoter']}\n")

/data4/miniforge3/envs/utr/lib/python3.10/site-packages/Bio/SeqFeature.py:230: BiopythonDeprecationWarning: Please use .location.strand rather than .strand
  warnings.warn(
100%|██████████| 4300/4300 [01:11<00:00, 60.00it/s]


In [ ]:
## 4. train a model using the predicted promoters
from transformers import AutoTokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

import torch
import pandas as pd
from datasets import Dataset

from transformers import DataCollatorForLanguageModeling
import warnings

# Define the arguments
base_model_name = "jinyuan22/promogen2-xsmall"
fasta_data_path = "promotercalculator_output/predicted_promoters_strong.fasta"
output_dir = "examples/promogen2_small_usp"
special_token = "<|usp|>"
per_device_train_batch_size = 32
learning_rate = 5e-5
warmup_steps = 1000
logging_steps = 1000
save_steps = 1000
eval_steps = 1000
max_steps = 10000
fp16 = True
seed = 42

# Set the seed
torch.manual_seed(seed)

model = GPT2LMHeadModel.from_pretrained(base_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": [special_token]})
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

def convert_fasta_to_csv(fasta_path, tag="<|usp|>"):
    fasta_dict = {}
    with open(fasta_path, "r") as f:
        for line in f:
            if line.startswith(">"):
                header = line.strip()
                fasta_dict[header] = ""
            else:
                fasta_dict[header] += line.strip()
    names = []
    seqs = []
    tags = []
    ids = []
    for k, v in fasta_dict.items():
        names.append(k)
        seqs.append(v)
        tags.append(tag)
        ids.append("|f")
        names.append(k)
        seqs.append(v[::-1])
        tags.append(tag)
        ids.append("|r")
    data = {"ID": ids, "Name": names, "Seq": seqs, "Tag": tags}
    df = pd.DataFrame(data)
    df.to_csv(fasta_path + "_traintext.csv", index=False)
    return df

usp_df = convert_fasta_to_csv(fasta_data_path, tag=special_token)
total_step = min([max_steps, int(len(usp_df) / per_device_train_batch_size) * 10])

if total_step < max_steps:
    # warnings
    warn_message = f"There are {len(usp_df)} samples in the dataset. The total steps will be set to {total_step}."
    warnings.warn(warn_message)
    warmup_steps = int(total_step * 0.1)
    logging_steps = int(total_step * 0.1)
    save_steps = int(total_step * 0.1)
    eval_steps = int(total_step * 0.1)

train_ds = Dataset.from_pandas(usp_df)


def preprocess_function(samples):
    processed_samples = {"input_ids": [], "attention_mask": []}
    for i, dna_sequence in enumerate((samples["Seq"])):
        tag = samples["Tag"][i]
        if samples["ID"][i] == "|f":
            tokenized_input = tokenizer(
                f"<|bos|>{tag}5{dna_sequence}3{tag}<|eos|>",
                padding="longest",
                truncation=True,
                max_length=204,
            )
        if samples["ID"][i] == "|r":
            tokenized_input = tokenizer(
                f"<|bos|>{tag}3{dna_sequence}5{tag}<|eos|>",
                padding="longest",
                truncation=True,
                max_length=204,
            )
        processed_samples["input_ids"].append(tokenized_input["input_ids"])
        processed_samples["attention_mask"].append(tokenized_input["attention_mask"])
    return processed_samples

tokenized_ds = train_ds.map(preprocess_function, batched=True, num_proc=8)

train_testvalid = tokenized_ds.train_test_split(test_size=0.1)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=f"{output_dir}_output",
    evaluation_strategy="steps",
    learning_rate=learning_rate,
    weight_decay=0.1,
    gradient_accumulation_steps=1,
    per_device_train_batch_size=per_device_train_batch_size,
    warmup_steps=warmup_steps,
    max_steps=total_step,
    logging_steps=logging_steps,
    eval_steps=eval_steps,
    logging_strategy="steps",
    save_steps=save_steps,
    report_to="tensorboard",
    fp16=fp16,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_testvalid["train"],
    eval_dataset=train_testvalid["test"],
    data_collator=data_collator,
)

trainer.train()

print(trainer.evaluate())

trainer.save_model(f"{output_dir}")
tokenizer.save_pretrained(f"{output_dir}")

In [ ]:
## 5. predict promoters using the trained model
from transformers import pipeline
model_path = output_dir
if fp16:
    model = GPT2LMHeadModel.from_pretrained(model_path, torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
else:
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

pipe = pipeline("text-generation", model=model, device=device, tokenizer=tokenizer)

@torch.no_grad()
def score(seq, tag, device="cuda:0"):
    if tag == 'none':
        inputs = tokenizer(f"<|bos|>5{seq}3<|eos|>", return_tensors="pt")
    else:
        inputs = tokenizer(f"<|bos|>{tag}5{seq}3{tag}<|eos|>", return_tensors="pt")
    # inputs = tokenizer(seq.upper(), return_tensors="pt")
    inputs.to(device)
    input_ids  = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    pred = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
    return pred['loss'].item()

txt = f"<|bos|>{special_token}5"
tag = special_token

num_return_sequences = 128*50
batch_size = 256
max_new_tokens = 60
repetition_penalty = 1.0
top_p = 1.0
temperature = 1.0
do_sample = True

all_outputs = []
for i in range(0, num_return_sequences, batch_size):
    # txt_batch = txt[i:i+args.batch_size]
    outputs = pipe(txt, 
            num_return_sequences=batch_size,
            max_new_tokens=max_new_tokens,
            repetition_penalty=repetition_penalty,
            top_p=top_p,
            temperature=temperature,
            do_sample=do_sample
        )
    all_outputs.extend(outputs)

model.eval().to(device)
seqs = [output["generated_text"].replace("<|bos|>", "").replace("5", "").replace("3", "").replace(tag, "") for output in all_outputs]
scores = [score(seq, tag) for seq in seqs]

sp_name = special_token.replace("<|", "").replace("|>", "")
output_file_name = f"examples/{sp_name}_t_{temperature}_r_{repetition_penalty}_p_{top_p}.fasta"

with open(output_file_name, "w") as f:
    for i, (seq, score) in enumerate(zip(seqs, scores)):
        f.write(f">{i}|score={score}\n{seq}\n")

In [ ]:
import itertools

def fasta_to_list(fasta_file):
    """Convert a fasta file to a list."""
    fasta_seqs = []
    with open(fasta_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                header = line[1:]
                fasta_seqs.append("")
            else:
                fasta_seqs[-1] += line
    return fasta_seqs

def generate_all_possible_kmers(k):
    """Generate all possible k-mers of a given length k for DNA using itertools."""
    bases = ['A', 'T', 'C', 'G']
    return [''.join(p) for p in itertools.product(bases, repeat=k)]

six_mers = generate_all_possible_kmers(6)

def count_kmers_in_sequences(sequences, k=6):
    """Count occurrences of all k-mers in a list of sequences."""
    all_kmers = generate_all_possible_kmers(k)
    kmers_count = {kmer: 0 for kmer in all_kmers}
    
    for seq in sequences:
        for i in range(len(seq) - k + 1):
            kmer = seq[i:i+k]
            if kmer in kmers_count:
                kmers_count[kmer] += 1

    return kmers_count

native_fasta_file = "promotercalculator_output/predicted_promoters_strong.fasta"
native_promoters = fasta_to_list(native_fasta_file)
native_promoters_kmer_count = count_kmers_in_sequences(native_promoters)

generate_promoter_file = output_file_name
generate_promoters = fasta_to_list(generate_promoter_file)
generate_promoters = [s for s in generate_promoters if len(s) == 60]
generate_promoters_kmer_count = count_kmers_in_sequences(generate_promoters[:len(native_promoters)])

In [ ]:
from scipy.stats import spearmanr, pearsonr
rho = spearmanr(list(native_promoters_kmer_count.values()), list(generate_promoters_kmer_count.values()))[0]
prs = pearsonr(list(native_promoters_kmer_count.values()), list(generate_promoters_kmer_count.values()))[0]
print(f"spearman: {rho:.3f}, pearson: {prs:.3f}")

In [ ]:
import seaborn as sns
import .pyplot as plt
import numpy as np
plt.figure(figsize=(5,5))
sns.regplot(x=np.array(list(native_promoters_kmer_count.values()))/np.sum(list(native_promoters_kmer_count.values())), 
            y=np.array(list(generate_promoters_kmer_count.values()))/np.sum(list(native_promoters_kmer_count.values())),
            scatter_kws={'s': 4}, 
            label=f"Spearman correlation: {rho:.4f}\nPearson correlation: {prs:.4f}")
plt.xlabel("Native promoters")
plt.ylabel("Generated promoters")
plt.legend()
